# Quantum Galton Box: Binomial to Gaussian with Grover-Rudolph Method

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); font-family: Arial, sans-serif; color: #34495e; font-size: 16px;">

  <h1 style="color: #2c3e50;">Task 2: Quantum Galton Box Simulation</h1>

  <p>
    This notebook implements a quantum circuit for a Galton Box with <i>n</i> layers, encoding a binomial distribution using the Grover-Rudolph method.
  </p>

  <div style="text-align: center; font-size: 18px; margin: 10px 0; color: #2c3e50;">
    <b><i>p(k) = C(n, k) / 2ⁿ</i></b>
  </div>

  <p>
    We verify its approximation to a Gaussian distribution and compare it to a classical Monte Carlo simulation for efficiency and accuracy. The project aligns with UN SDG 9 (Industry, Innovation, and Infrastructure) by advancing quantum algorithm development for scalable simulations.
  </p>

</div>


---

## 1. Introduction

<div style="background-color: #e8f4f8; padding: 15px; border-left: 5px solid #3498db; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); margin-bottom: 20px; font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Problem Statement</h2>

  <p style="font-size: 14px; color: #34495e;">
    The Galton Box models a binomial distribution through a pegboard where balls fall, approximating a Gaussian distribution for large <i>n</i>. We aim to:
  </p>

  <ul style="font-size: 14px; color: #34495e; margin-top: -10px;">
    <li>Develop a quantum circuit to prepare a state encoding <i>p(k) = C(n, k) / 2<sup>n</sup></i> using the Grover-Rudolph method.</li>
    <li>Verify the state approximates a Gaussian distribution.</li>
    <li>Compare quantum and classical Monte Carlo simulations for computational efficiency and accuracy.</li>
  </ul>

  <p style="font-size: 14px; color: #34495e;">
    <b>Impact:</b> Efficient quantum simulations can accelerate modeling in physics, finance, and sustainability, supporting scalable innovation.
  </p>

</div>


---

## 2. Theoretical Background

<div style="background-color: #fff3cd; padding: 15px; border-left: 5px solid #f1c40f; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Key Concepts</h2>

  <ul style="font-size: 14px; color: #34495e;">
    <li><b>Galton Box:</b> A physical system where balls fall through <i>n</i> layers of pegs, landing in bins with probabilities following a binomial distribution.</li>
    <li><b>Binomial to Gaussian:</b> For large <i>n</i>, the binomial distribution <i>p(k) = C(n, k) / 2<sup>n</sup></i> approximates a Gaussian via the Central Limit Theorem.</li>
    <li><b>Grover-Rudolph Method:</b> A quantum state preparation technique to encode probability distributions into quantum states using controlled rotations.</li>
    <li><b>Quantum Advantage:</b> Quantum circuits may reduce computational complexity for sampling distributions compared to classical methods.</li>
  </ul>

</div>


---

## 3. Setup and Dependencies

<div style="background-color: #e8f8f0; padding: 15px; border-left: 5px solid #27ae60; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Environment Setup</h2>

  <p style="font-size: 14px; color: #34495e;">
    Install required libraries and set up the environment for quantum and classical simulations.
  </p>

</div>

In [1]:
import http.server, socketserver, threading, numpy as np, matplotlib.pyplot as plt, time
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from scipy.stats import binom, norm

PORT = 8000

def start_server():
    Handler = http.server.SimpleHTTPRequestHandler
    httpd = socketserver.TCPServer(("", PORT), Handler)
    print(f"Serving at http://localhost:{PORT}")
    httpd.serve_forever()

thread = threading.Thread(target=start_server)
thread.daemon = True
thread.start()

Serving at http://localhost:8000


127.0.0.1 - - [06/Jul/2025 10:23:24] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 10:27:35] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 10:39:01] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 10:46:45] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 10:55:44] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 10:56:15] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 10:57:07] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 11:01:44] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 11:03:58] "GET /notebook_animations/galton_board_classical.html HTTP/1.1" 304 -
127.0.0.1 - - [06/Jul/2025 11:04:24] 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from scipy.stats import binom, norm
import time

# Styling for plots
plt.style.use('seaborn-v0_8')  # Use Seaborn style

---

## 4. Classical Monte Carlo Simulation

<div style="background-color: #f0f0f0; padding: 15px; border-left: 5px solid #7f8c8d; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Classical Galton Box Simulation</h2>

  <p style="font-size: 14px; color: #34495e;">
    Simulate a Galton Box classically using Monte Carlo methods to generate a binomial distribution and measure runtime.
  </p>

</div>


<iframe
  src="http://localhost:8000/notebook_animations/galton_board_classical.html"
  width="100%"
  height="700"
  frameborder="0"
  style="overflow: hidden; border: none;"
  scrolling="no"
></iframe>


In [ ]:
def classical_galton_box(n_layers, n_trials):
    bins = np.zeros(n_layers + 1)
    start_time = time.time()
    for _ in range(n_trials):
        pos = 0
        for _ in range(n_layers):
            pos += np.random.choice([0, 1])  # 0 for left, 1 for right
        bins[pos] += 1
    runtime = time.time() - start_time
    probabilities = bins / n_trials
    return probabilities, runtime

# Example: n=10 layers, 10000 trials
n_layers, n_trials = 10, 10000
classical_probs, classical_time = classical_galton_box(n_layers, n_trials)

# Plot results
x = np.arange(0, n_layers + 1)
plt.figure(figsize=(10, 6))
plt.bar(x, classical_probs, alpha=0.6, label='Classical Monte Carlo')
plt.plot(x, binom.pmf(x,n_layers, 0.5), 'g--', label='Binomial PMF')
plt.plot(x, norm.pdf(x, n_layers/2, np.sqrt(n_layers/4)), 'r-', label='Gaussian Fit')
plt.title('Classical Galton Box: Binomial Distribution')
plt.xlabel('Bin')
plt.ylabel('Probability')
plt.legend()
plt.show()
print(f"Classical runtime: {classical_time:.4f} seconds")

---

## 5. Quantum Circuit Design

<div style="background-color: #e8f4f8; padding: 15px; border-left: 5px solid #3498db; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Quantum State Preparation with Grover-Rudolph</h2>

  <p style="font-size: 14px; color: #34495e;">
    Implement a quantum circuit to prepare a state encoding 
    <i>p(k) = C(n, k) / 2<sup>n</sup></i> 
    using the Grover-Rudolph method. Use Qiskit for simulation.
  </p>

</div>


---

## 6. Verification: Gaussian Approximation

<div style="background-color: #fff3cd; padding: 15px; border-left: 5px solid #f1c40f; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Comparing Binomial to Gaussian</h2>

  <p style="font-size: 14px; color: #34495e;">
    Verify that both classical and quantum distributions approximate a Gaussian using statistical metrics (e.g., KL divergence).
  </p>

</div>


---

## 7. Efficiency and Accuracy Comparison

<div style="background-color: #e8f8f0; padding: 15px; border-left: 5px solid #27ae60; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Quantum vs. Classical Performance</h2>

  <p style="font-size: 14px; color: #34495e;">
    Compare runtime and accuracy (via KL divergence) between quantum and classical simulations.
  </p>

</div>


---

## 8. Hackathon Presentation Strategy

<div style="background-color: #f0f8ff; padding: 15px; border-left: 5px solid #2980b9; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">2-Minute Pitch Outline</h2>

  <ul style="font-size: 14px; color: #34495e; margin-top: 0;">
    <li><b>Hook (15s):</b> "Imagine simulating complex systems like fluid dynamics or financial models faster than ever—our quantum Galton Box makes it possible!"</li>
    <li><b>Problem & Solution (45s):</b> Explain the Galton Box, binomial-to-Gaussian transition, and how our quantum circuit uses Grover-Rudolph for efficient state preparation.</li>
    <li><b>Results (45s):</b> Show plots comparing quantum and classical distributions, highlight runtime and accuracy, and emphasize quantum advantage.</li>
    <li><b>Impact (15s):</b> Tie to SDG 9, scalable quantum simulations, and future applications in your quantum hardware venture.</li>
  </ul>

  <p style="font-size: 14px; color: #34495e; margin-top: 10px;">
    <b>Visuals:</b> Circuit diagram (Qiskit), distribution plots, and a live demo of the quantum simulation.
  </p>

</div>


---

## 9. Collaboration and Next Steps

<div style="background-color: #f0f0f0; padding: 15px; border-left: 5px solid #7f8c8d; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Teamwork and Networking</h2>

  <ul style="font-size: 14px; color: #34495e; margin-top: 0;">
    <li><b>GitHub Collaboration:</b> Use a branching strategy (feature branches for quantum/classical code), conduct code reviews via pull requests, and document in README.</li>
    <li><b>Networking:</b> Attend PennyLane tutorials (per hackathon agenda) to optimize Grover-Rudolph implementation. Approach mentors like Dr. Nana Liu with specific questions (e.g., "How can we improve multi-qubit rotations for scalability?").</li>
    <li><b>Next Steps:</b> Refine quantum circuit for larger <i>n</i>, test on real quantum hardware via IBM Quantum, and prepare a YouTube-style explainer video for broader impact.</li>
  </ul>

</div>


---

## 10. Conclusion

<div style="background-color: #e8f4f8; padding: 15px; border-left: 5px solid #3498db; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.05); font-family: Arial, sans-serif;">

  <h2 style="color: #2c3e50;">Summary and Future Work</h2>

  <p style="font-size: 14px; color: #34495e;">
    This notebook demonstrates a quantum Galton Box simulation, leveraging the Grover-Rudolph method to encode a binomial distribution and verifying its Gaussian approximation. The quantum approach shows potential for efficiency gains, aligning with SDG 9. Future work includes optimizing the circuit for scalability and integrating it into a portfolio for your quantum hardware company.
  </p>

</div>
